In [137]:
import torch
import torch.nn as nn
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')
parent_path = 'C:\\Users\\SIU856533724\\OneDrive - Southern Illinois University\\Research\\Paper\\Social Networks\\Trend Prediction\\Comment-Section-UI\\UI\\angular-interactive-comments-section\\flask\\data'

In [138]:

file_name = 'fox-new-york-could-face'
file = os.path.join(parent_path, f'{file_name}.csv')
df = pd.read_csv(file)

In [139]:
df.head(5)

,Unnamed: 0,comment,time,username,id,timestamp,time_diff
0,321,And so ... the 'lawful' cheating begins!,"22 Dec, 2023 03:28 AM",senorbiggles,322,2023-12-22 03:28:00,0 days 00:00:00
1,875,And so ... the 'lawful' cheating begins!,"22 Dec, 2023 03:28 AM",senorbiggles,876,2023-12-22 03:28:00,0 days 00:00:00
2,296,Democrats desperately rigging the elections in...,"22 Dec, 2023 03:28 AM",HarryBaals404,297,2023-12-22 03:28:00,0 days 00:00:00
3,850,Democrats desperately rigging the elections in...,"22 Dec, 2023 03:28 AM",HarryBaals404,851,2023-12-22 03:28:00,0 days 00:00:00
4,144,Democrats can’t win on their achievements so t...,"22 Dec, 2023 03:29 AM",HarryBaals404,145,2023-12-22 03:29:00,0 days 00:01:00


In [140]:
list(df.iloc[1][1:])

["And so ... the 'lawful' cheating begins!",
 '22 Dec, 2023 03:28 AM',
 'senorbiggles',
 876,
 '2023-12-22 03:28:00',
 '0 days 00:00:00']

In [141]:
def get_so_far_comments(ith_comment):
    # row_index = df.loc[df['id'] == comment_id].index[0]
    ids = df.iloc[0:ith_comment+1]['id']
    return list(ids)


In [142]:
# get_so_far_comments(1)

In [143]:
df_trend = pd.read_csv(os.path.join(parent_path, f'{file_name}-trends.csv'))

In [144]:
df_trend.shape

(1064, 10)

In [145]:
trends = ["approval","toxic","obscene", 'insult', "threat", "hate", "offensive", "neither"]

In [146]:
def get_single_trend_score():
    trendData = {}
    for label in trends:
        trendData[label] = list(df_trend[label])

    return trendData

In [147]:
previous_emotion = []

In [148]:
# import random

# def get_emotion(escores, id):
#     # print("id")
#     if id == 0:
#         previous_emotion = escores

#     if id%100 == 0:
#         X = escores[27]*0.6
#         escores[27] = escores[27] - X
#         y = X*(random.sample(range(0, 100), 1)[0]/100)
#         random_numbers = random.sample(range(0, 27), 3)
#         escores[random_numbers[0]] = X*0.2
#         escores[random_numbers[1]] = X*0.7
#         escores[random_numbers[2]] = X*0.1
#         previous_emotion = escores
#         return escores
    
#     if id%200 == 0:
#         X = escores[27]*0.6
#         escores[27] = escores[27] - X
#         y = X*(random.sample(range(0, 100), 1)[0]/100)
#         random_numbers = random.sample(range(0, 27), 3)
#         escores[random_numbers[0]] = X*0.1
#         escores[random_numbers[1]] = X*0.6
#         escores[random_numbers[2]] = X*0.3
#         previous_emotion = escores
#         return escores
    
#     return previous_emotion


In [149]:
from common_function import get_emotions, get_sentiment_dict
from score_aggregation import get_polarity_score, find_indices

emotions = list(get_emotions().values())
sentiment_dict = get_sentiment_dict()

In [150]:
pos_indices = find_indices(sentiment_dict['positive'], emotions)
neg_indices = find_indices(sentiment_dict['negative'], emotions)
amb_indices = find_indices(sentiment_dict['ambiguous'], emotions)
neu_indices = find_indices(sentiment_dict['neutral'], emotions)

def get_emotion(escores, id):
    # print(escores)

    if id < 100:
        X = escores[27]*0.4
        escores[27] = escores[27] - X
        for i in amb_indices:
            escores[i] += (X*0.10)

    elif id > 100 and id < 200:
        X = escores[27]*0.5
        escores[27] = escores[27] - X
        for i in pos_indices:
            escores[i] += (X*0.15)

    elif id > 200 and id < 400:
        X = escores[27]*0.6
        escores[27] = escores[27] - X
        for i in pos_indices:
            escores[i] += (X*0.10)

    elif id > 400 and id < 600:
        X = escores[27]*0.6
        escores[27] = escores[27] - X
        for i in pos_indices:
            escores[i] += (X*0.15)

    elif id > 600 and id < 800:
        X = escores[27]*0.65
        escores[27] = escores[27] - X
        for i in neg_indices:
            escores[i] += (X*0.10)

    elif id > 800:
        X = escores[27]*0.7
        escores[27] = escores[27] - X
        for i in neg_indices:
            escores[i] += (X*0.15)

    # print(escores)
    escores =  [float(i)/sum(escores) for i in escores]

    # print("SUM:", sum(escores))
    return escores

In [151]:
def get_trends(tscores, id):

    # print(escores)
    if id < 100:
        X = tscores[7]*0.5
        tscores[7] = tscores[7] - X
        for i in [0, 7]:
            tscores[i] += (X*0.10)

    elif id > 100 and id < 200:
        X = tscores[7]*0.57
        tscores[7] = tscores[7] - X
        for i in [0, 7]:
            tscores[i] += (X*0.10)

    elif id > 200 and id < 500:
        X = tscores[7]*0.6
        tscores[7] = tscores[7] - X
        for i in [0, 7]:
            tscores[i] += (X*0.10)

    elif id > 500 and id < 700:
        X = tscores[7]*0.5
        tscores[7] = tscores[7] - X
        for i in [1, 2, 3, 5]:
            tscores[i] += (X*0.10)

    elif id > 700 and id < 800:
        X = tscores[7]*0.6
        tscores[7] = tscores[7] - X
        for i in [1, 2, 3, 5]:
            tscores[i] += (X*0.10)

    elif id > 800 and id < 900:
        X = tscores[7]*0.65
        tscores[7] = tscores[7] - X
        for i in [4, 6]:
            tscores[i] += (X*0.10)

    elif id > 900:
            X = tscores[7]*0.7
            tscores[7] = tscores[7] - X
            for i in [4, 6]:
                tscores[i] += (X*0.10)
    # print(escores)
    tscores =  [float(i)/sum(tscores) for i in tscores]

    # print("SUM:", sum(escores))
    return tscores

In [152]:
from score_aggregation import get_emotion_trends_polarity

def getData():
    print("Get All Emotion, Trend, Sentiment Data")
    emotionData = []
    trendData = []
    sentimentData = []
    for index in range(df.shape[0]):
        ids = get_so_far_comments(ith_comment=index)
        # print("Index: ", index)
        # print("So far Comments: ", ids)
        escores, tscores, polarity = get_emotion_trends_polarity(ids)
        
        escores = get_emotion(escores, index)
        tscores = get_trends(tscores, index)
        polarity = get_polarity_score(escores)

        emotionData.append(escores)
        trendData.append(tscores)
        sentimentData.append(polarity)

    print(emotionData)

    return emotionData, trendData, sentimentData

In [153]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import json

app = Flask(__name__)
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})

@app.route('/api/new-comment', methods=['POST'])
def welcome():
    comment = request.json['content']
    img = request.json['img']
    isYou = request.json['isYou']
    score = request.json['score']
    username = request.json['username']
    print("new: ", comment)
    return 'Successfully comment added!'

@app.route('/api/data/all-comments', methods=['GET'])
def getApiData():
    escores, tscores, polarity = getData()
    # print(escores)
    return json.dumps({"emotion": escores, "trend": tscores, "sentiment": polarity})

@app.route('/api/data/single-trend/all', methods=['GET'])
def getSingleTrend():
    # return get_single_trend_score()
    # print(get_single_trend_score())
    return json.dumps(get_single_trend_score())

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
